# Comprehensive Stock Fundamental Analysis

This notebook performs an in-depth analysis of a stock by examining:
1. Fundamental financial data
2. Key financial ratios and metrics
3. Technical indicators
4. News and geopolitical context (via OpenAI)
5. AI-generated summary report

## Setup and Dependencies

First, let's install and import all required packages. We'll use uv for dependency management.

In [2]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import requests
from datetime import datetime, timedelta
import pytz  # For timezone handling
import json
from dotenv import load_dotenv
import openai

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_theme(style="darkgrid")

# Load environment variables for API keys
load_dotenv()

True

## OpenAI API Configuration

In [3]:
# Set up OpenAI client
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    print("Warning: OPENAI_API_KEY not found in environment variables.")
try:
    client = openai.OpenAI(api_key=openai_api_key)
    print("OpenAI client initialized successfully.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")

OpenAI client initialized successfully.


## Input Stock Symbol

Enter the stock symbol you want to analyze.

In [4]:
# Input stock symbol
stock_symbol = "FSLR" #input("Enter stock symbol (e.g., AAPL, MSFT, GOOGL): ").strip().upper()

# Set analysis timeframes
end_date = datetime.now()

start_date_2y = end_date - timedelta(days=2*365)  # 2 years of data
start_date_1y = end_date - timedelta(days=365)    # 1 year of data

print(f"Analyzing {stock_symbol} from {start_date_2y.date()} to {end_date.date()}")

Analyzing FSLR from 2023-04-20 to 2025-04-19


## 1. Data Collection

Let's gather historical price data, company information, and financial statements.

In [5]:
def get_stock_data(symbol, start_date, end_date):
    """Fetch historical stock data"""
    try:
        stock = yf.Ticker(symbol)
        # Use tz_localize=None to get timezone-naive datetimes in the result
        data = stock.history(start=start_date, end=end_date, auto_adjust=True)
        data.index = data.index.tz_localize(None)
        if data.empty:
            return None, f"No data found for symbol {symbol}"
        return stock, data
    except Exception as e:
        return None, f"Error fetching data: {e}"

# Fetch stock data
stock, hist_data_2y = get_stock_data(stock_symbol, start_date_2y, end_date)

if isinstance(hist_data_2y, str):
    print(hist_data_2y)  # Print error message
else:
    print(f"Successfully retrieved data for {stock_symbol}")
    hist_data_1y = hist_data_2y[hist_data_2y.index >= start_date_1y]
    
    # Display the stock information
    info = stock.info
    company_name = info.get('longName', stock_symbol)
    sector = info.get('sector', 'N/A')
    industry = info.get('industry', 'N/A')
    
    print(f"Company: {company_name}")
    print(f"Sector: {sector}")
    print(f"Industry: {industry}")
    print(f"Current Price: ${info.get('currentPrice', 'N/A')}")

Successfully retrieved data for FSLR
Company: First Solar, Inc.
Sector: Technology
Industry: Solar
Current Price: $127.98


In [6]:
# Get financial data

# Balance Sheet
balance_sheet_annual = stock.balance_sheet
balance_sheet_quarterly = stock.quarterly_balance_sheet

# Income Statement
income_stmt_annual = stock.income_stmt
income_stmt_quarterly = stock.quarterly_income_stmt

# Cash Flow
cash_flow_annual = stock.cashflow
cash_flow_quarterly = stock.quarterly_cashflow

print("Financial data fetched successfully.")

# Display some key financial metrics from the most recent annual data
print("\nRecent Annual Financial Highlights (in millions USD):")
if not income_stmt_annual.empty:
    recent_year = income_stmt_annual.columns[0]
    revenue = income_stmt_annual.loc['Total Revenue', recent_year] / 1e6
    net_income = income_stmt_annual.loc['Net Income', recent_year] / 1e6
    print(f"Revenue: ${revenue:.2f}M")
    print(f"Net Income: ${net_income:.2f}M")
    
    if 'EBITDA' in income_stmt_annual.index:
        ebitda = income_stmt_annual.loc['EBITDA', recent_year] / 1e6
        print(f"EBITDA: ${ebitda:.2f}M")

if not balance_sheet_annual.empty:
    recent_year = balance_sheet_annual.columns[0]
    total_assets = balance_sheet_annual.loc['Total Assets', recent_year] / 1e6
    total_liabilities = balance_sheet_annual.loc['Total Liabilities Net Minority Interest', recent_year] / 1e6
    print(f"Total Assets: ${total_assets:.2f}M")
    print(f"Total Liabilities: ${total_liabilities:.2f}M")
    print(f"Equity: ${(total_assets - total_liabilities):.2f}M")

Financial data fetched successfully.

Recent Annual Financial Highlights (in millions USD):
Revenue: $4206.29M
Net Income: $1292.04M
EBITDA: $1868.71M
Total Assets: $12124.36M
Total Liabilities: $4146.78M
Equity: $7977.58M


## 2. Financial Ratio Analysis

Calculate and visualize key financial ratios that are important for fundamental analysis.

In [7]:
def calculate_financial_ratios(stock, info):
    """Calculate important financial ratios for fundamental analysis"""
    ratios = {}
    
    # Valuation Ratios
    ratios['P/E Ratio'] = info.get('trailingPE', 'N/A')
    ratios['Forward P/E'] = info.get('forwardPE', 'N/A')
    ratios['P/B Ratio'] = info.get('priceToBook', 'N/A')
    ratios['P/S Ratio'] = info.get('priceToSalesTrailing12Months', 'N/A')
    ratios['EV/EBITDA'] = info.get('enterpriseToEbitda', 'N/A')
    
    # PEG Ratio (Price/Earnings to Growth)
    pe = info.get('trailingPE', None)
    earnings_growth = info.get('earningsGrowth', None)
    if isinstance(pe, (int, float)) and isinstance(earnings_growth, (int, float)) and earnings_growth != 0:
        ratios['PEG Ratio'] = pe / (earnings_growth * 100) # if earnings_growth > 1 else pe / earnings_growth
    else:
        ratios['PEG Ratio'] = 'N/A'
    
    # Profitability Ratios
    ratios['Gross Margin'] = info.get('grossMargins', 'N/A')
    ratios['Operating Margin'] = info.get('operatingMargins', 'N/A')
    ratios['Net Profit Margin'] = info.get('profitMargins', 'N/A')
    ratios['ROE'] = info.get('returnOnEquity', 'N/A')
    ratios['ROA'] = info.get('returnOnAssets', 'N/A')
    
    # Liquidity Ratios
    ratios['Current Ratio'] = info.get('currentRatio', 'N/A')
    ratios['Quick Ratio'] = info.get('quickRatio', 'N/A')
    
    # Debt Ratios
    ratios['Debt-to-Equity'] = info.get('debtToEquity', 'N/A')
    ratios['Interest Coverage'] = info.get('interestCoverage', 'N/A')
    
    # Dividend Ratios
    ratios['Dividend Yield'] = info.get('dividendYield', 'N/A')
    ratios['Payout Ratio'] = info.get('payoutRatio', 'N/A')
    
    # Growth Rates
    ratios['Revenue Growth (YoY)'] = info.get('revenueGrowth', 'N/A')
    ratios['Earnings Growth (YoY)'] = info.get('earningsGrowth', 'N/A')
    
    return ratios

financial_ratios = calculate_financial_ratios(stock, stock.info)

# Display financial ratios
print("\nKey Financial Ratios:")

ratios_df = pd.DataFrame(list(financial_ratios.items()), columns=['Ratio', 'Value'])
# Convert numeric values and format percentages
ratios_df['Value'] = pd.to_numeric(ratios_df['Value'], errors='ignore')

# Format values for display
formatted_values = []
for idx, row in ratios_df.iterrows():
    if row['Ratio'] in ['Dividend Yield', 'Gross Margin', 'Operating Margin', 'Net Profit Margin', 'ROE', 'ROA', 'Payout Ratio',
                        'Revenue Growth (YoY)', 'Earnings Growth (YoY)']:
        if isinstance(row['Value'], (int, float)):
            formatted_values.append(f"{row['Value']:.2%}")
        else:
            formatted_values.append(row['Value'])
    elif row['Ratio'] == 'PEG Ratio':
        if isinstance(row['Value'], (int, float)):
            formatted_values.append(f"{row['Value']:.2f}")
        else:
            formatted_values.append(row['Value'])
    else:
        if isinstance(row['Value'], (int, float)):
            formatted_values.append(f"{row['Value']:.2f}")
        else:
            formatted_values.append(row['Value'])
            
ratios_df['Formatted Value'] = formatted_values
display(ratios_df[['Ratio', 'Formatted Value']])


Key Financial Ratios:


/var/folders/w2/1nyptw5136b9mqhj4dvgdky00000gn/T/ipykernel_9811/1965853841.py:52: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  ratios_df['Value'] = pd.to_numeric(ratios_df['Value'], errors='ignore')


,Ratio,Formatted Value
0,P/E Ratio,10.65
1,Forward P/E,6.14
2,P/B Ratio,1.72
3,P/S Ratio,3.26
4,EV/EBITDA,6.95
5,PEG Ratio,0.84
6,Gross Margin,44.17%
7,Operating Margin,30.17%
8,Net Profit Margin,30.72%
9,ROE,17.62%


## 3. Historical Price and Volume Analysis

In [8]:
def safe_plot_display(figure):
    """Safely display a plotly figure, with fallback to a static image if nbformat is missing"""
    try:
        # Try to display the interactive plot
        figure.show()
    except ValueError as e:
        if "nbformat" in str(e):
            # If nbformat is missing or outdated, switch to a non-interactive display
            print("Warning: Interactive plot could not be displayed due to missing or outdated nbformat.")
            print("Displaying static plot instead. Run `!pip install nbformat>=4.2.0` and restart kernel for interactive plots.")
            # Generate a static image
            img_bytes = figure.to_image(format="png")
            from IPython.display import Image, display
            display(Image(img_bytes))
        else:
            # If it's another error, re-raise it
            raise


# Create an interactive price and volume chart
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.05, 
                    subplot_titles=('Price', 'Volume'),
                    row_heights=[0.7, 0.3])

# Price candlestick
fig.add_trace(
    go.Candlestick(
        x=hist_data_2y.index,
        open=hist_data_2y['Open'],
        high=hist_data_2y['High'],
        low=hist_data_2y['Low'],
        close=hist_data_2y['Close'],
        name="Price"
    ),
    row=1, col=1
)

# Add moving averages
hist_data_2y['MA50'] = hist_data_2y['Close'].rolling(window=50).mean()
hist_data_2y['MA200'] = hist_data_2y['Close'].rolling(window=200).mean()

fig.add_trace(
    go.Scatter(
        x=hist_data_2y.index,
        y=hist_data_2y['MA50'],
        name="50-Day MA",
        line=dict(color='orange', width=1)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=hist_data_2y.index,
        y=hist_data_2y['MA200'],
        name="200-Day MA",
        line=dict(color='purple', width=1)
    ),
    row=1, col=1
)

# Volume
colors = ['red' if row['Open'] > row['Close'] else 'green' for i, row in hist_data_2y.iterrows()]
fig.add_trace(
    go.Bar(
        x=hist_data_2y.index,
        y=hist_data_2y['Volume'],
        name="Volume",
        marker_color=colors
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title=f"{stock_symbol} Historical Price and Volume (2 Years)",
    xaxis_rangeslider_visible=False,
    height=800,
    width=1500,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Show the figure with fallback handling
# safe_plot_display(fig)

## 4. Technical Analysis Indicators

In [9]:
def calculate_technical_indicators(data):
    """Calculate technical indicators for the stock"""
    # Make a copy to avoid modifying the original dataframe
    df = data.copy()
    
    # Moving Averages
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()
    
    # Exponential Moving Averages
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    
    # MACD
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    
    # Relative Strength Index (RSI)
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    df['BB_Std'] = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
    df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)
    
    # Average True Range (ATR)
    df['TR1'] = abs(df['High'] - df['Low'])
    df['TR2'] = abs(df['High'] - df['Close'].shift())
    df['TR3'] = abs(df['Low'] - df['Close'].shift())
    df['TR'] = df[['TR1', 'TR2', 'TR3']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=14).mean()
    
    return df

# Calculate price trend for the last 3, 6, and 12 months
def calculate_price_trends(hist_data, as_of_date=None):
    '''
    Calculate percentage price change for the last 3, 6, and 12 months.
    Returns a dict with keys '3m', '6m', '12m'.
    '''
    if as_of_date is None:
        as_of_date = hist_data.index[-1]
    trends = {}
    periods = {
        '3m': 90,
        '6m': 180,
        '12m': 365
    }
    for label, days in periods.items():
        past_date = as_of_date - pd.Timedelta(days=days)
        # Find the closest available date in the index
        past_idx = hist_data.index.searchsorted(past_date)
        if past_idx >= len(hist_data):
            trends[label] = None
            continue
        past_price = hist_data.iloc[past_idx]['Close']
        current_price = hist_data.loc[as_of_date]['Close']
        if pd.isna(past_price) or pd.isna(current_price):
            trends[label] = None
        else:
            trends[label] = f"{100 * (current_price - past_price) / past_price:.2f}%"

    # All-time high and low
    all_time_high = hist_data['Close'].max()
    all_time_high_date = hist_data['Close'].idxmax()
    all_time_low = hist_data['Close'].min()
    all_time_low_date = hist_data['Close'].idxmin()
    
        # Format output string
    output = []
    if trends.get('3m') is not None:
        output.append(f"3-month price change: {trends['3m']} (Change in closing price over the last 3 months)")
    if trends.get('6m') is not None:
        output.append(f"6-month price change: {trends['6m']} (Change in closing price over the last 6 months)")
    if trends.get('12m') is not None:
        output.append(f"12-month price change: {trends['12m']} (Change in closing price over the last 12 months)")
    output.append(f"All-time high: ${all_time_high:.2f} on {all_time_high_date.date()} (Highest closing price in available data)")
    output.append(f"All-time low: ${all_time_low:.2f} on {all_time_low_date.date()} (Lowest closing price in available data)")
    return "\n".join(output)

# Example usage:
price_trends = calculate_price_trends(hist_data_1y)
print('Price trends (percent change):', price_trends)


# Calculate technical indicators
tech_indicators = calculate_technical_indicators(hist_data_1y)



# Plot the technical indicators
# 1. RSI
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05,
                    subplot_titles=('Price with Bollinger Bands', 'RSI', 'MACD'),
                    row_heights=[0.5, 0.25, 0.25])

# Price with Bollinger Bands
fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['Close'],
        name="Close Price",
        line=dict(color='blue')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['BB_Upper'],
        name="BB Upper",
        line=dict(color='gray', width=1)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['BB_Middle'],
        name="BB Middle",
        line=dict(color='gray', width=1, dash='dash')
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['BB_Lower'],
        name="BB Lower",
        line=dict(color='gray', width=1)
    ),
    row=1, col=1
)

# RSI
fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['RSI'],
        name="RSI",
        line=dict(color='purple')
    ),
    row=2, col=1
)

# Add RSI overbought/oversold lines
fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=[70] * len(tech_indicators),
        name="Overbought",
        line=dict(color='red', width=1, dash='dash')
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=[30] * len(tech_indicators),
        name="Oversold",
        line=dict(color='green', width=1, dash='dash')
    ),
    row=2, col=1
)

# MACD
fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['MACD'],
        name="MACD",
        line=dict(color='blue')
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=tech_indicators.index,
        y=tech_indicators['MACD_Signal'],
        name="Signal Line",
        line=dict(color='red')
    ),
    row=3, col=1
)

# MACD Histogram
colors = ['green' if val >= 0 else 'red' for val in tech_indicators['MACD_Histogram']]
fig.add_trace(
    go.Bar(
        x=tech_indicators.index,
        y=tech_indicators['MACD_Histogram'],
        name="MACD Histogram",
        marker_color=colors
    ),
    row=3, col=1
)

# Update layout
fig.update_layout(
    title=f"{stock_symbol} Technical Indicators (1 Year)",
    height=900,
    width=1500,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Add y-axis titles
fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="RSI", row=2, col=1)
fig.update_yaxes(title_text="MACD", row=3, col=1)

# Show the figure with fallback handling
safe_plot_display(fig)

Price trends (percent change): 3-month price change: -33.65% (Change in closing price over the last 3 months)
6-month price change: -34.79% (Change in closing price over the last 6 months)
12-month price change: -26.89% (Change in closing price over the last 12 months)
All-time high: $300.71 on 2024-06-12 (Highest closing price in available data)
All-time low: $120.38 on 2025-04-08 (Lowest closing price in available data)


## 5. News and Geopolitical Analysis

Let's fetch and analyze recent news about the company using OpenAI.

In [ ]:
def get_news_analysis(company_name, stock_symbol, client):
    """Get news analysis from OpenAI"""
    if client is None:
        return "OpenAI client not initialized. Please set up your API key."
    
    try:
        # Current date for context
        today = datetime.now().strftime("%d %B %Y")
        prompt = f"""
                Today is {today}. Provide a comprehensive summary of the most significant recent news about {company_name} ({stock_symbol}) 
                that could impact its stock price and fundamental value. The user will provide you also with the price trend of the company. 
                
                Focus on:
                
                1. Recent earnings reports and financial performance.
                2. Reasoning behind the price development of the stock in the past.  
                2. Major business developments (new products, services, markets)
                3. Future innovations, partnerships, or acquisitions. 
                4. Leadership changes or organizational restructuring
                5. Regulatory developments affecting the company
                6. Macroeconomic factors influencing the company and its industry
                7. Geopolitical events that might impact operations or supply chains
                8. Competitive landscape changes

                You should do a detailed internet search for each of the points above before reaching a conclusion.

                Think in steps.
                
                Format your response as a well-structured analysis with clear sections and bullet points where appropriate.
                Include dates of key events where possible. Highlight the potential impact of each development on the company's future performance.

                {price_trends}
                """
            
        response = client.responses.create(
            model="gpt-4o-mini",
            tools=[{ "type": "web_search_preview" }],
            input=prompt,
        )
        return response.output[1].content[0].text
    except Exception as e:
        return f"Error fetching news analysis: {e}"


company_name = stock.info.get('longName', stock_symbol)
news_analysis = get_news_analysis(company_name, stock_symbol, client)
print(f"\nNews and Geopolitical Analysis for {company_name} ({stock_symbol}):\n")
display(Markdown(news_analysis))


News and Geopolitical Analysis for First Solar, Inc. (FSLR):

**Comprehensive Analysis of Recent Developments Impacting First Solar, Inc. (FSLR)**

As of April 18, 2025, First Solar, Inc. (FSLR) has experienced several significant developments across various facets of its operations. Below is a detailed analysis of these developments, categorized as per the specified points:

---

**1. Recent Earnings Reports and Financial Performance**

- **Fourth Quarter and Full-Year 2024 Financial Results**:
  - *Net Sales*: Achieved $4.2 billion in 2024, marking a 27% increase from the previous year. The fourth quarter contributed $1.5 billion to this total.
  - *Net Income*: Reported earnings per diluted share of $12.02 for the full year and $3.65 for the fourth quarter.
  - *Net Cash Position*: Ended 2024 with a net cash balance of $1.2 billion, up from $0.7 billion in the third quarter, primarily due to proceeds from Section 45X tax credits and module segment operating cash flows.
  - *2025 Gu

## 6. Comprehensive AI-Generated Analysis Report

Now, let's generate a comprehensive report that analyzes all the data we've collected.

In [ ]:
def generate_comprehensive_report(stock_symbol, stock_info, financial_ratios, tech_indicators, news_analysis, client, price_trends=None):
    """Generate a comprehensive stock analysis report using OpenAI"""
    if client is None:
        return "OpenAI client not initialized. Please set up your API key."
    
    try:
        # Prepare financial ratios string
        ratios_str = "\n".join([f"{ratio}: {value}" for ratio, value in financial_ratios.items()])
        
        # Get key technical indicators values for the most recent date
        recent_tech = {}
        if tech_indicators is not None and not tech_indicators.empty:
            recent_date = tech_indicators.index[-1]
            recent_tech = {
                'Price': tech_indicators.loc[recent_date, 'Close'],
                'RSI': tech_indicators.loc[recent_date, 'RSI'],
                'MACD': tech_indicators.loc[recent_date, 'MACD'],
                'MACD Signal': tech_indicators.loc[recent_date, 'MACD_Signal'],
                'ATR': tech_indicators.loc[recent_date, 'ATR'],
                'BB Upper': tech_indicators.loc[recent_date, 'BB_Upper'],
                'BB Middle': tech_indicators.loc[recent_date, 'BB_Middle'],
                'BB Lower': tech_indicators.loc[recent_date, 'BB_Lower']
            }
        
        tech_str = "\n".join([f"{indicator}: {value}" for indicator, value in recent_tech.items()])
        
        # Add price trend summary
        
        
        # Prepare info summary
        info_summary = f"""
        Company: {stock_info.get('longName', stock_symbol)}
        Symbol: {stock_symbol}
        Sector: {stock_info.get('sector', 'N/A')}
        Industry: {stock_info.get('industry', 'N/A')}
        Current Price: ${stock_info.get('currentPrice', 'N/A')}
        52-Week Range: ${stock_info.get('fiftyTwoWeekLow', 'N/A')} - ${stock_info.get('fiftyTwoWeekHigh', 'N/A')}
        Market Cap: ${stock_info.get('marketCap', 'N/A')}
        Beta: {stock_info.get('beta', 'N/A')}
        Price Trends: {price_trends}
        """
        
        # Current date for context
        today = datetime.now().strftime("%Y-%m-%d")
        
        prompt = f"""
        Today is {today}. As a financial analyst, provide a comprehensive investment analysis report for {stock_symbol} based on the following information:
        
        COMPANY INFORMATION:
        {info_summary}
        
        FINANCIAL RATIOS:
        {ratios_str}
        
        TECHNICAL INDICATORS (Most Recent):
        {tech_str}
        
        NEWS ANALYSIS:
        {news_analysis}
        
        Based on all this information, provide a comprehensive investment analysis with the following sections:
        
        1. Executive Summary - A brief overview of the company and its current situation
        2. Fundamental Analysis - Analysis of financial health, valuation, and growth prospects
        3. Technical Analysis - Interpretation of price movements and technical indicators
        4. News Impact Analysis - How recent news affects the company's prospects
        5. Risk Assessment - Key risks facing the company
        6. Investment Outlook - Overall assessment including strengths, weaknesses, opportunities, and threats
        7. Recommendation - Clear investment recommendation (Buy/Hold/Sell) with reasoning
        
        The report should be well-structured with clear sections and bullet points where appropriate.
        Focus on providing actionable insights based on the data provided.
        """
        
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Use appropriate model based on your OpenAI subscription
            messages=[
                {"role": "system", "content": "You are a senior financial analyst with extensive experience in equity research and investment analysis."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=2500
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error generating comprehensive report: {e}"


print("\nGenerating comprehensive investment analysis report...\n")
comprehensive_report = generate_comprehensive_report(
    stock_symbol,
    stock.info,
    financial_ratios,
    tech_indicators,
    news_analysis,
    client,
    price_trends=price_trends
)

In [12]:
# This cell demonstrates how to render markdown in the notebook (for preview),
# but for PDF export, use the Makefile command: `make export-pdf`
from IPython.display import Markdown, display
display(Markdown(comprehensive_report))

# Investment Analysis Report for First Solar, Inc. (FSLR)

## 1. Executive Summary
First Solar, Inc. (FSLR) is a leading player in the solar energy sector, focusing on the manufacturing of photovoltaic (PV) solar modules and providing utility-scale PV power plants. As of April 18, 2025, the company is trading at $127.98, down significantly from its all-time high of $300.71 in June 2024. Despite recent price declines, First Solar has reported strong financial performance, driven by increased net sales and a robust cash position. The company is strategically expanding its manufacturing capabilities and investing in innovative technologies to enhance its competitive edge.

## 2. Fundamental Analysis
### Financial Health
- **Market Capitalization**: $13.72 billion
- **P/E Ratio**: 10.65, indicating a relatively low valuation compared to historical norms for growth stocks.
- **Forward P/E**: 6.14, suggesting potential for earnings growth.
- **Gross Margin**: 44.17%, indicating strong profitability.
- **Operating Margin**: 30.17%, reflecting efficient operations.
- **Net Profit Margin**: 30.72%, showcasing effective cost management.

### Valuation
- **P/B Ratio**: 1.72, suggesting the stock is trading above its book value, which is typical for growth companies.
- **PEG Ratio**: 0.84, indicating that the stock is undervalued relative to its earnings growth potential.
- **Debt-to-Equity Ratio**: 9.01, which is extremely high, indicating significant leverage that could pose risks.

### Growth Prospects
- **Revenue Growth (YoY)**: 30.7%, indicating strong demand and market expansion.
- **Earnings Growth (YoY)**: 12.7%, reflecting solid operational performance.
- **2025 Guidance**: Projected net sales of $5.3 billion to $5.8 billion and EPS of $17.00 to $20.00, indicating continued growth.

## 3. Technical Analysis
- **Current Price**: $127.98
- **52-Week Range**: $116.56 - $306.77, indicating high volatility.
- **3-Month Price Change**: -33.65%, suggesting bearish sentiment.
- **RSI**: 50.50, indicating the stock is neither overbought nor oversold.
- **MACD**: -3.19, indicating bearish momentum.
- **Bollinger Bands**: The price is currently near the middle band, suggesting potential for a rebound or further decline.

## 4. News Impact Analysis
Recent news highlights several positive developments:
- **Financial Performance**: Strong Q4 2024 results and a solid cash position due to tax credits.
- **Manufacturing Expansion**: New facilities in Alabama and Louisiana will enhance production capacity.
- **Technological Advancements**: Innovations in module technology and strategic acquisitions position the company for future growth.
- **Regulatory Support**: The Inflation Reduction Act (IRA) provides incentives that benefit First Solar, enhancing its market position.

## 5. Risk Assessment
- **High Leverage**: The company's debt-to-equity ratio is extremely high, which could pose risks in a rising interest rate environment.
- **Supply Chain Issues**: Challenges in the supply chain and labor shortages could impact production and project timelines.
- **Market Volatility**: The solar industry is subject to fluctuations in demand and pricing, which could affect profitability.
- **Regulatory Risks**: Changes in government policies or incentives could impact the company's financial performance.

## 6. Investment Outlook
### Strengths
- Strong financial performance with significant revenue and earnings growth.
- Strategic expansion and technological innovation enhance competitive positioning.
- Robust cash position bolstered by tax credits.

### Weaknesses
- High leverage raises financial risk.
- Recent stock price volatility may deter conservative investors.

### Opportunities
- Expansion into new markets and increased manufacturing capacity.
- Continued innovation in solar technology could lead to market leadership.

### Threats
- Supply chain disruptions and labor shortages could hinder growth.
- Competitive pressures from both domestic and international manufacturers.

## 7. Recommendation
**Recommendation: Buy**

**Rationale**:
- The stock is currently undervalued based on its PEG ratio and forward P/E, suggesting potential for price appreciation.
- Strong growth prospects, driven by robust financial performance and strategic initiatives, position First Solar favorably in the solar market.
- The company's proactive approach to leveraging regulatory incentives and expanding manufacturing capabilities enhances its long-term growth potential.

Investors should consider accumulating shares at the current price, taking advantage of the dip while recognizing the inherent risks associated with high leverage and market volatility.

## 7. Export Report to PDF (Optional)

You can save the comprehensive report to a PDF file for sharing or future reference.

In [18]:
import markdown2
import tempfile
import os

def export_report_to_pdf(report_text, filename="stock_analysis_report.pdf"):
    """Export the markdown report to a PDF file using FPDF"""
    try:
        from fpdf import FPDF
        
        class PDF(FPDF):
            def header(self):
                company_name = stock.info.get('longName', stock_symbol)
                self.set_font('Arial', 'B', 15)
                self.cell(0, 10, f"{company_name} ({stock_symbol}) - Stock Analysis Report", 0, 1, 'C')
                self.ln(10)
            
            def footer(self):
                self.set_y(-15)
                self.set_font('Arial', 'I', 8)
                self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')
    
        # Create PDF instance
        pdf = PDF()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.add_page()
        
        # Process the markdown content
        current_font_size = 12
        in_list = False
        
        for line in report_text.split('\n'):
            # Safely encode the line to avoid encoding issues
            line = line.encode('latin-1', 'replace').decode('latin-1')
            
            # Section headers
            if line.strip().startswith('# '):
                pdf.set_font('Arial', 'B', 16)
                pdf.cell(0, 10, line.replace('#', '').strip(), ln=True)
                pdf.ln(5)
                current_font_size = 12
            elif line.strip().startswith('## '):
                pdf.set_font('Arial', 'B', 14)
                pdf.cell(0, 10, line.replace('##', '').strip(), ln=True)
                pdf.ln(3)
                current_font_size = 12
            elif line.strip().startswith('### '):
                pdf.set_font('Arial', 'B', 13)
                pdf.cell(0, 10, line.replace('###', '').strip(), ln=True)
                current_font_size = 12
            elif line.strip().startswith('- ') or line.strip().startswith('* '):
                # List items - using a hyphen instead of bullet point
                pdf.set_font('Arial', '', current_font_size)
                item_text = line.strip()[2:].strip()
                pdf.cell(10, 7, '-', 0, 0)  # Use hyphen instead of bullet point
                pdf.multi_cell(0, 7, item_text)
                in_list = True
            elif line.strip().startswith('---'):
                pdf.ln(2)
                pdf.line(pdf.get_x(), pdf.get_y(), pdf.get_x() + 190, pdf.get_y())
                pdf.ln(5)
            elif line.strip() == '':
                # Empty line
                pdf.ln(5)
                in_list = False
            else:
                # Regular text
                if in_list and line.strip() and (line[0] == ' ' or line[0] == '\t'):
                    # Continuation of list item with indentation
                    pdf.set_font('Arial', '', current_font_size)
                    pdf.set_x(pdf.get_x() + 10)
                    pdf.multi_cell(0, 7, line.strip())
                else:
                    in_list = False
                    # Handle bold text (very simplified approach)
                    if '**' in line:
                        parts = line.split('**')
                        for i, part in enumerate(parts):
                            if i % 2 == 0:  # Even parts are regular text
                                pdf.set_font('Arial', '', current_font_size)
                                pdf.write(7, part)
                            else:  # Odd parts are bold
                                pdf.set_font('Arial', 'B', current_font_size)
                                pdf.write(7, part)
                        pdf.ln()
                    else:
                        pdf.set_font('Arial', '', current_font_size)
                        pdf.multi_cell(0, 7, line)
        
        pdf.output(filename)
        print(f"Report successfully exported to {filename}")
    except Exception as e:
        print(f"Error in PDF generation: {e}")
        print("Try installing FPDF2 for better UTF-8 support: pip install fpdf2")

# Export the comprehensive report
export_report_to_pdf(comprehensive_report)

Report successfully exported to stock_analysis_report.pdf
